## Demo: Get S&P stock data from Yahoo or Quandl

### Define the imports

In [4]:
#!pip install pandas-datareader
#!pip install quandl

In [5]:
import bs4 as bs
import datetime as dt
import os
import pandas as pd
import pandas_datareader.data as web
import pickle
import requests
import quandl


### Specify global variables

In [6]:
data_path = 'Stocks_Data'
sp500_ticker_url = 'http://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
ticker_cache_filename = 'sp500tickers.pickle'
ticker_cache_path = '{}/{}'.format(data_path, ticker_cache_filename)
my_quandl_api_key = 'sRoopcCR-jfCKimYgz65'
tickers_of_interest = ['GOOGL', 'AMZN', 'AAPL', 'ORCL', 'MSFT']

### Find,scrap and save the list of S&P 500 tickers
- Tickers are scraped from wikipedia url: 'http://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
- Ticker is then pickled (cached)

In [7]:
def save_sp500_tickers():
    resp = requests.get(sp500_ticker_url)
    soup = bs.BeautifulSoup(resp.text, 'lxml')
    table = soup.find('table', {'class': 'wikitable sortable'})
    tickers = []
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text
        tickers.append(ticker)
        
    with open(ticker_cache_path,"wb") as f:
        pickle.dump(tickers,f)
    
    print('S&P 500 ticker have been sourced from: {0} and been cached here: {1}'.format(sp500_ticker_url, ticker_cache_path))
        
    return tickers

In [8]:
#tickers = save_sp500_tickers()

### Get the S & P Index from 2000-01-01 to 2018-0-01

In [35]:
def getSP500PriceIndecies(start_date = "2000-01-01", end_date = "2018-01-01"):
    #quandl.get("BCIP/_INX", authtoken="sRoopcCR-jfCKimYgz65", start_date=start_date, end_date=end_date)
    #return quandl.get("CME/SPU2017", authtoken="sRoopcCR-jfCKimYgz65", start_date=start_date, end_date=end_date)
    return quandl.get_table('WIKI/PRICES', date='1999-11-18', ticker='^GSPC')

sp500_data = getSP500PriceIndecies()

In [36]:
sp500_data.head(10)

,ticker,date,open,high,low,close,volume,ex-dividend,split_ratio,adj_open,adj_high,adj_low,adj_close,adj_volume
None,,,,,,,,,,,,,,


### Now we will use pandas_datareader api to get the stock data for each s&p 500 ticker fom Yahoo

In [14]:
def getDataFromYahoo(reload_sp500=False, data_source='quandl'):
    
    if reload_sp500:
        tickers = save_sp500_tickers()
    else:
        with open(ticker_cache_path,"rb") as f:
            tickers = pickle.load(f)
    
    if not os.path.exists(data_path):
        os.makedirs(data_path)

    start = dt.datetime(2000, 1, 1)
    end = dt.datetime(2017, 12, 31)
    
    for ticker in tickers:
        # just in case your connection breaks, we'd like to save our progress!
        ticker_cache_loc = '{0}/{1}.csv'.format(data_path, ticker)
        if not os.path.exists(ticker_cache_loc) and (ticker in tickers_of_interest):            
            print("Currently sourcing data for ticker: {0} from source: {1}..\n".format(ticker, data_source))
            if data_source == 'yahoo':
                # Use Yahoo
                df = web.DataReader(ticker, "yahoo", start, end)                
            else:
                # Use Quandl
                quandl.ApiConfig.api_key = my_quandl_api_key
                df = quandl.get("WIKI/{}".format(ticker), start_date=start, end_date=end)
            df.to_csv(ticker_cache_loc)            
            print("Souced data will be cached here: {}\n".format(ticker_cache_loc))
        else:
            print('Already have ticker: {} or it is not required!'.format(ticker))


In [16]:
#getDataFromYahoo()

### Read the ticker data for each tech company into a Pandas dataframe 

In [64]:
def createDataframePerTicker():
    tables = {}
    main_table = pd.DataFrame()
    for count,ticker in enumerate(tickers_of_interest):
        df = pd.read_csv('{0}/{1}.csv'.format(data_path, ticker))
        df.set_index('Date', inplace=True)
        
        # Compute daily Open/Close and High/Low percentage difference
        df['{}_HL_pct_diff'.format(ticker)] = (df['High'] - df['Low']) / df['Low']
        df['{}_OC_pct_diff'.format(ticker)] = (df['Close'] - df['Open']) / df['Open']
        
        # Also rename the 'Adj. Close' column as ticker value
        df.rename(columns={'Adj. Close':ticker}, inplace=True)
        
        # Also rename the 'Adj. Volume' column as 'Adj_Volume'
        df.rename(columns={'Adj. Volume':'{}_Adj_Volume'.format(ticker)}, inplace=True)
        
        # Also remove the unwanted colums: ['Open','High','Low','Close', 'Ex-Dividend', 'Split Ratio', 'Adj. Open', 'Adj. High', 'Adj. Low']
        df.drop(['Open','High','Low','Close', 'Volume','Ex-Dividend', 'Split Ratio', 'Adj. Open', 'Adj. High', 'Adj. Low'],1,inplace=True)
        tables[ticker] = df
        
        # Join the individual ticker tables into a main table
        if main_table.empty:
            main_table = df
        else:
            main_table = main_table.join(df, how='outer')
    return main_table, tables
            

In [65]:
#tables = createDataframePerTicker()
main_table, tables = createDataframePerTicker()

### Lag the returns of the stock price

In [70]:
def lagStockReturns(tables, lag = 5):
    new_tables = {}
    for ticker, df in tables.items():
        return_col = '{}_OC_pct_diff'.format(ticker)
        lag_back_col = 'Lag_bwd_'
        lag_fwd_col = 'Lag_fwd_'
        volume_col = 'Adj_Volume'
        for i in range(1, lag, 1):
            df.loc[:,'{0}{1} '.format(lag_back_col, str(i))] = df[return_col].shift(i)
        new_df = df[[x for x in df.columns if lag_back_col in x or x == return_col or volume_col in x]].iloc[lag:,]
        new_tables[ticker] = new_df
    return new_tables

tables_new = lagStockReturns(tables, lag = 5)

In [71]:
tables_new['GOOGL']

,GOOGL_Adj_Volume,GOOGL_OC_pct_diff,Lag_1,Lag_2,Lag_3,Lag_4
Date,,,,,,
2004-08-26,7094800.0,0.028204,0.011837,-0.057264,-0.012279,0.072270
2004-08-27,6211700.0,-0.018039,0.028204,0.011837,-0.057264,-0.012279
2004-08-30,5196700.0,-0.031060,-0.018039,0.028204,0.011837,-0.057264
2004-08-31,4917800.0,0.000489,-0.031060,-0.018039,0.028204,0.011837
2004-09-01,9138200.0,-0.023856,0.000489,-0.031060,-0.018039,0.028204
2004-09-02,15118600.0,0.024422,-0.023856,0.000489,-0.031060,-0.018039
2004-09-03,5152400.0,-0.009312,0.024422,-0.023856,0.000489,-0.031060
2004-09-07,5847500.0,0.005643,-0.009312,0.024422,-0.023856,0.000489
2004-09-08,4985600.0,0.015485,0.005643,-0.009312,0.024422,-0.023856


In [53]:
#tables['GOOGL']
#main_table['2004-08-01':]
#main_table.index

,GOOGL,GOOGL_Adj_Volume,GOOGL_HL_pct_diff,GOOGL_OC_pct_diff,AMZN,AMZN_Adj_Volume,AMZN_HL_pct_diff,AMZN_OC_pct_diff,AAPL,AAPL_Adj_Volume,AAPL_HL_pct_diff,AAPL_OC_pct_diff,ORCL,ORCL_Adj_Volume,ORCL_HL_pct_diff,ORCL_OC_pct_diff,MSFT,MSFT_Adj_Volume,MSFT_HL_pct_diff,MSFT_OC_pct_diff
Date,,,,,,,,,,,,,,,,,,,,
2004-08-02,NaN,NaN,NaN,NaN,38.30,5912200.0,0.025126,-0.000522,2.029235,91273000.0,0.034372,0.012829,9.642539,40242900.0,0.039690,0.026949,19.152267,52267000.0,0.013849,0.008843
2004-08-03,NaN,NaN,NaN,NaN,37.61,7785600.0,0.049691,-0.017503,2.010601,52907400.0,0.018299,-0.006667,9.543131,44096400.0,0.015209,-0.000946,18.850075,53990900.0,0.015000,-0.010575
2004-08-04,NaN,NaN,NaN,NaN,37.12,9765600.0,0.029121,0.007054,2.042729,69122200.0,0.030478,0.019237,9.796169,51684300.0,0.042776,0.026515,18.843360,46217900.0,0.007539,0.002107
2004-08-05,NaN,NaN,NaN,NaN,35.61,8402400.0,0.059356,-0.047810,2.017026,61125400.0,0.033600,-0.011339,9.615428,46577100.0,0.029412,-0.017544,18.487445,55591700.0,0.025073,-0.021677
2004-08-06,NaN,NaN,NaN,NaN,35.49,11513100.0,0.052224,0.006809,1.913573,123072600.0,0.047138,-0.036246,9.208760,38588500.0,0.034347,-0.023011,18.225545,75628000.0,0.029564,-0.009489
2004-08-09,NaN,NaN,NaN,NaN,35.32,9167200.0,0.041607,-0.000566,1.946986,72711800.0,0.021469,0.015075,9.371427,31533900.0,0.026471,0.004845,18.252406,51877500.0,0.006642,-0.002569
2004-08-10,NaN,NaN,NaN,NaN,37.10,10070400.0,0.046414,0.039798,2.025380,87759000.0,0.039209,0.037183,9.579280,31955300.0,0.029098,0.014354,18.615037,57632700.0,0.018349,0.015385
2004-08-11,NaN,NaN,NaN,NaN,36.56,9448100.0,0.036517,0.004396,1.992609,80598000.0,0.028751,-0.002894,9.208760,43650500.0,0.032706,-0.020192,18.406860,53097300.0,0.011397,0.001096
2004-08-12,NaN,NaN,NaN,NaN,36.29,9118100.0,0.032624,0.002209,1.951484,56550200.0,0.018824,-0.002955,8.946686,56499100.0,0.042945,-0.018831,18.050945,50279700.0,0.016754,-0.013216


In [21]:
# Define which online source one should use
data_source = 'google'

# We would like all available data from 01/01/2000 until 12/31/2016.
start_date = '2000-01-01'
end_date = '2018-01-01'

tickers = ['SPY']

# User pandas_reader.data.DataReader to load the desired data. As simple as that.
panel_data = web.DataReader(tickers, data_source, start_date, end_date)

panel_data.head(5)

RemoteDataError: No data fetched using 'GoogleDailyReader'